In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [9]:
%cd "/content/drive/MyDrive/Image Binarization/novelty"

/content/drive/MyDrive/Image Binarization/novelty


In [4]:
import cv2
import os
import numpy as np



In [8]:
pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 64.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [5]:
import segmentation_models_pytorch as smp

In [6]:

pip install torch

In [7]:
import torch

In [11]:


# Function to divide the input image into patches
def divide_into_patches(image_path):
    # Read the image using OpenCV
    original_image = cv2.imread(image_path)

    # Get the original width and height
    original_height, original_width, _ = original_image.shape

    # Calculate the required padding for the right and bottom sides
    pad_right = 256 - (original_width % 256) if original_width % 256 != 0 else 0
    pad_bottom = 256 - (original_height % 256) if original_height % 256 != 0 else 0

    # Pad the image with white color only on the right and bottom sides
    padded_image = cv2.copyMakeBorder(original_image, 0, pad_bottom, 0, pad_right, cv2.BORDER_CONSTANT, value=(255, 255, 255))

    # Cut the image into 256x256 patches
    patches = []
    for i in range(0, padded_image.shape[0], 256):
        for j in range(0, padded_image.shape[1], 256):
            patch = padded_image[i:i+256, j:j+256]
            patches.append(patch)

    return patches, original_width, original_height

# Function to apply the model parameters to each patch and obtain predictions
def apply_model_to_patches(model, patches):
    predictions = []
    for patch in patches:
        # Convert patch to tensor and unsqueeze to add batch dimension
        patch_tensor = torch.tensor(patch).permute(2, 0, 1).unsqueeze(0).float()

        # Apply model to patch
        with torch.no_grad():
            prediction = model(patch_tensor)

        # Append prediction to list
        predictions.append(prediction.squeeze(dim=0).cpu().numpy())

    return predictions

# Function to reconstruct the total image from the predictions of all patches
def reconstruct_image(predictions, original_width, original_height):
    # Determine the dimensions of the stitched image
    cols = (original_width + 255) // 256  # Calculate the number of columns
    rows = (original_height + 255) // 256  # Calculate the number of rows
    width = cols * 256
    height = rows * 256

    # Create a blank canvas to stitch the images onto
    stitched_image = 255 * np.ones((height, width, 1), dtype=np.uint8)

    # Reconstruct the image from predictions of all patches
    idx = 0
    for i in range(rows):
        for j in range(cols):
            if idx < len(predictions):
                y = i * 256
                x = j * 256
                stitched_image[y:y+256, x:x+256] = (torch.tensor(predictions[idx]).permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
                idx += 1

    # Crop the stitched image to the original dimensions
    cropped_stitched_image = stitched_image[:original_height, :original_width].copy()
    return cropped_stitched_image

# Example usage:
input_image_path = "input.png"  # Provide the path to your input image
model_name_to_load = "BCE_FPN_epoch60.pth"  # Provide the path to your model file

# Load the model
model = smp.FPN(
            encoder_name='resnet34',
            in_channels=3,
            classes=1,
            activation='sigmoid',
            encoder_weights='imagenet'
        )
model.load_state_dict(torch.load(model_name_to_load, map_location=torch.device('cpu')))
model.eval()

# Divide the image into patches
patches, original_width, original_height = divide_into_patches(input_image_path)

# Apply the model parameters to each patch and obtain predictions
predictions = apply_model_to_patches(model, patches)

# Reconstruct the total image from the predictions of all patches
reconstructed_image = reconstruct_image(predictions, original_width, original_height)

# Save the reconstructed image
output_image_path = "output.png"  # Provide the path to save the reconstructed image
cv2.imwrite(output_image_path, reconstructed_image)

print("Original Image Dimensions:", original_width, "x", original_height)
print("Reconstructed Image Dimensions:", reconstructed_image.shape[1], "x", reconstructed_image.shape[0])
print(f"Reconstructed image saved as {output_image_path}")

Original Image Dimensions: 9371 x 843
Reconstructed Image Dimensions: 9371 x 843
Reconstructed image saved as output.png
